In [424]:
from dotenv import load_dotenv
import os
import requests
import json
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
import pandas as pd
import geopandas as gpd
from pymongo import MongoClient
import time
from cartoframes.viz import Map, Layer, popup_element

In [356]:
load_dotenv()

True

In [357]:
token = os.getenv("token")
client_id = os.getenv("client_id")
client_secret = os.getenv("client_secret")

In [4]:
#token, client_id, client_secret

In [408]:
client = MongoClient("localhost:27017")
client.list_database_names()
db = client["Ironhack"]
companies = db.get_collection("companies")
companies

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'Ironhack'), 'companies')

In [413]:
query_sf_1 = {"offices.0.city": "San Francisco", "funding_rounds.0.raised_amount": {"$gte": 1000000}}
projection_sf_1 = {"_id": 0, "name": 1, "category_code": 1, "founded_year": 1, "total_money_raised": 1, "offices": 1}

san_francisco = list(companies.find(query_sf_1, projection_sf_1))

for i in san_francisco:
    i.update({'latitude': i['offices'][0]['latitude'], 'longitude': i['offices'][0]['longitude']})
    del i['offices']

#san_francisco
df_sf_pre = pd.DataFrame(san_francisco)
df_sf=df_sf_pre.dropna(subset=['latitude', 'longitude'], thresh=1)


query_sf_2 = {"offices.0.city": "San Francisco", "category_code": "design"}
projection_sf_2 = {"name": 1, "_id": 0}

df_sf_design = pd.DataFrame(list(companies.find(query_sf_2, projection_sf_2)))

display(df_sf, df_sf_design)

,name,category_code,founded_year,total_money_raised,latitude,longitude
0,Digg,news,2004.0,$45M,37.764726,-122.394523
1,Twitter,social,2006.0,$1.16B,37.776805,-122.416924
2,StumbleUpon,web,2002.0,$18.5M,37.775196,-122.419204
3,Powerset,search,2006.0,$22.5M,37.778613,-122.395289
4,Technorati,advertising,2002.0,$32.1M,37.779558,-122.393041
...,...,...,...,...,...,...
286,UTOPY,software,1999.0,$7.5M,37.767575,-122.411157
287,Siluria Technologies,nanotech,2008.0,$103M,37.448491,-122.180281
288,ZoomSystems,hardware,2002.0,$87M,37.781689,-122.391061
290,TeachScape,software,1999.0,$16M,37.786905,-122.404392


,name
0,99designs


In [414]:
query_bgl_1 = {"offices.0.city": "Bangalore", "funding_rounds.0.raised_amount": {"$gte": 1000000}}
projection_bgl_1 = {"_id": 0, "name": 1, "category_code": 1, "founded_year": 1, "total_money_raised": 1, "offices": 1}

bangalore = list(companies.find(query_bgl_1, projection_bgl_1))


for i in bangalore:
    i.update({'latitude': i['offices'][0]['latitude'], 'longitude': i['offices'][0]['longitude']})
    del i['offices']
    

df_bgl_pre = pd.DataFrame(bangalore)
#df_bgl
df_bgl=df_bgl_pre.dropna(subset=['latitude', 'longitude'], thresh=1)


query_bgl_2 = {"offices.0.city": "Bangalore", "category_code": "design"}
projection_bgl_2 = {"name": 1, "_id": 0}
df_bgl_design = pd.DataFrame(list(companies.find(query_bgl_2, projection_bgl_2)))


display(df_bgl, df_bgl_design)

,name,category_code,founded_year,total_money_raised,latitude,longitude
0,Minglebox,web,2006,$7M,12.971606,77.594376
3,Guruji,search,2006,$15M,12.937677,77.565740
4,Ziva Software,mobile,2005,$1.7M,12.973784,77.643590
6,Manthan Systems,analytics,2003,$47M,12.970895,77.597675
7,Must See India,travel,2007,$5M,12.908470,77.600339
8,Althea Systems,hardware,2009,$3M,12.934960,77.613685


""


In [417]:
query_stk_1 = {"offices.0.city": "Stockholm", "funding_rounds.0.raised_amount": {"$gte": 1000000}}
projection_stk_1 = {"_id": 0, "name": 1, "category_code": 1, "founded_year": 1, 
                    "total_money_raised": 1, "offices": 1}

stockholm = list(companies.find(query_stk_1, projection_stk_1))


for i in stockholm:
    i.update({'latitude': i['offices'][0]['latitude'], 'longitude': i['offices'][0]['longitude']})
    del i['offices']
    

df_stk_pre = pd.DataFrame(stockholm)
#df_stk
df_stk=df_stk_pre.dropna(subset=['latitude', 'longitude'], thresh=1)


query_stk_2 = {"offices.0.city": "Stockholm", "category_code": "design"}
projection_stk_2 = {"name": 1, "_id": 0}
df_stk_design = pd.DataFrame(list(companies.find(query_stk_2, projection_stk_2)))

display(df_stk, df_stk_design)

,name,category_code,founded_year,total_money_raised,latitude,longitude
0,TestFreaks,ecommerce,2006,$5.8M,59.333991,18.057120
1,PacketFront,software,2001,€26M,59.332788,18.064488
2,Stardoll,games_video,2004,$10M,59.345553,18.037611
3,Bambuser,mobile,2007,$2.5M,59.332788,18.064488
6,Momail,mobile,2005,$5.4M,59.335981,18.075506
7,Syntensia,software,2005,$3.5M,59.612126,17.894767


""


In [418]:
df_count = pd.DataFrame({"San Francisco 1M$": [len(df_sf)], "San Francisco Design": [len(df_sf_design)], 
                         "Bangalore 1M$": [len(df_bgl)], "Bangalore Design": [len(df_bgl_design)],
                         "Stockhol 1M$": [len(df_sf)], "Stockholm Design": [len(df_stk_design)]})
df_count = df_count.set_index(["San Francisco 1M$", "San Francisco Design", 
                               "Bangalore 1M$", "Bangalore Design", 
                               "Stockhol 1M$", "Stockholm Design"])
df_count

,,,,,
San Francisco 1M$,San Francisco Design,Bangalore 1M$,Bangalore Design,Stockhol 1M$,Stockholm Design
239,1,6,0,239,0


In [147]:
sf_map = folium.Map(location=[df_sf_ll['latitude'][0], df_sf_ll['longitude'][0]], zoom_start=14)

for i, row in df_sf_ll.iterrows():
    folium.Marker(location=[row['latitude'], row['longitude']], popup=row['name']).add_to(sf_map)

#sf_map

In [495]:
sf_lat = 37.7730148
sf_long = -122.4284791

sf_icon = folium.Icon(color="red", icon_color="yellow", icon="map-pin", prefix="fa")

map_sf = folium.Map(location=[sf_lat, sf_long], zoom_start=14)

folium.Marker(location=[sf_lat, sf_long], icon=sf_icon).add_to(map_sf)

map_sf.save("mp_sf.html")
map_sf

In [434]:
#StarBuck

url = "https://api.foursquare.com/v3/places/search?ll=37.77%2C-122.42&radius=5000&chains=ab4c54c0-d68a-012e-5619-003048cad9da&limit=50"
headers = {"Authorization": token , "accept": "application/json" }

response = requests.get(url, headers=headers)

list_star_sf=response.json()

latitudes = []
longitudes = []
names = []

for i in list_star_sf['results']:
    latitude = i['geocodes']['main']['latitude']
    latitudes.append(latitude)
    longitude = i['geocodes']['main']['longitude']
    longitudes.append(longitude)
    name = i['chains'][0]['name'] + ', ' + i['location']['address']
    names.append(name)
    
df_star_sf = pd.DataFrame ({'names': names, 'latitudes': latitudes, 'longitudes': longitudes})
#df_star_sf 

In [458]:
#Education 

url = "https://api.foursquare.com/v3/places/search?ll=37.77%2C-122.42&radius=5000&categories=12009&limit=50"
headers = {"Authorization": token , "accept": "application/json" }

response = requests.get(url, headers=headers)

list_school_sf=response.json()

latitudes = []
longitudes = []
names = []

for i in list_school_sf['results']:
    latitude = i['geocodes']['main']['latitude']
    latitudes.append(latitude)
    longitude = i['geocodes']['main']['longitude']
    longitudes.append(longitude)
    name = i['categories'][0]['name'] + ', ' + i['location']['address']
    names.append(name)
    
df_school_sf = pd.DataFrame ({'names': names, 'latitudes': latitudes, 'longitudes': longitudes})
#df_school_sf

In [445]:
#Airport

url = "https://api.foursquare.com/v3/places/search?ll=37.77%2C-122.42&radius=5000&categories=19031&limit=50"
headers = {"Authorization": token , "accept": "application/json" }

response = requests.get(url, headers=headers)

list_airport_sf=response.json()

latitudes = []
longitudes = []
names = []

for i in list_airport_sf['results']:
    latitude = i['geocodes']['main']['latitude']
    latitudes.append(latitude)
    longitude = i['geocodes']['main']['longitude']
    longitudes.append(longitude)
    name = i['categories'][0]['name'] + ', ' + i['location']['address']
    names.append(name)
    
df_airport_sf = pd.DataFrame ({'names': names, 'latitudes': latitudes, 'longitudes': longitudes})
#df_airport_sf 

In [448]:
#Party

url = "https://api.foursquare.com/v3/places/search?ll=37.77%2C-122.42&radius=5000&categories=13003&limit=50"
headers = {"Authorization": token , "accept": "application/json" }

response = requests.get(url, headers=headers)

list_party_sf=response.json()

latitudes = []
longitudes = []
names = []

for i in list_party_sf['results']:
    latitude = i['geocodes']['main']['latitude']
    latitudes.append(latitude)
    longitude = i['geocodes']['main']['longitude']
    longitudes.append(longitude)
    name = i['categories'][0]['name'] + ', ' + i['location']['address']
    names.append(name)
    
df_party_sf = pd.DataFrame ({'names': names, 'latitudes': latitudes, 'longitudes': longitudes})
#df_party_sf 

In [450]:
#Vegan Places

url = "https://api.foursquare.com/v3/places/search?ll=37.77%2C-122.42&radius=5000&categories=13377&limit=50"
headers = {"Authorization": token , "accept": "application/json" }

response = requests.get(url, headers=headers)

list_vegan_sf=response.json()

latitudes = []
longitudes = []
names = []

for i in list_vegan_sf['results']:
    latitude = i['geocodes']['main']['latitude']
    latitudes.append(latitude)
    longitude = i['geocodes']['main']['longitude']
    longitudes.append(longitude)
    name = i['categories'][0]['name'] + ', ' + i['location']['address']
    names.append(name)
    
df_vegan_sf = pd.DataFrame ({'names': names, 'latitudes': latitudes, 'longitudes': longitudes})
#df_vegan_sf 

In [456]:
#Basketball Court

url = "https://api.foursquare.com/v3/places/search?ll=37.77%2C-122.42&radius=5000&categories=18008&limit=50"
headers = {"Authorization": token , "accept": "application/json" }

response = requests.get(url, headers=headers)

list_court_sf=response.json()

latitudes = []
longitudes = []
names = []

for i in list_court_sf['results']:
    latitude = i['geocodes']['main']['latitude']
    latitudes.append(latitude)
    longitude = i['geocodes']['main']['longitude']
    longitudes.append(longitude)
    name = i['categories'][0]['name'] + ', ' + i['location']['address']
    names.append(name)
    
df_court_sf = pd.DataFrame ({'names': names, 'latitudes': latitudes, 'longitudes': longitudes})
#df_court_sf 

In [457]:
#Pets Hairdresser

url = "https://api.foursquare.com/v3/places/search?ll=37.77%2C-122.42&radius=5000&categories=15054&limit=50"
headers = {"Authorization": token , "accept": "application/json" }

response = requests.get(url, headers=headers)

list_pet_sf=response.json()

latitudes = []
longitudes = []
names = []

for i in list_pet_sf['results']:
    latitude = i['geocodes']['main']['latitude']
    latitudes.append(latitude)
    longitude = i['geocodes']['main']['longitude']
    longitudes.append(longitude)
    name = i['categories'][0]['name'] + ', ' + i['location']['address']
    names.append(name)
    
df_pet_sf = pd.DataFrame ({'names': names, 'latitudes': latitudes, 'longitudes': longitudes})
#df_pet_sf

In [496]:
df_sf_places = pd.concat([df_star_sf, df_school_sf, df_airport_sf, 
                          df_party_sf, df_vegan_sf, df_court_sf, df_pet_sf])

mean_lat_sf = df_sf_places['latitudes'].mean()
center_lat_sf = round(mean_lat_sf, 7)
mean_lon_sf = df_sf_places['longitudes'].mean()
center_lon_sf = round(mean_lon_sf, 7)

center_icon = folium.Icon(color="orange", icon_color="black", icon="star", prefix="fa")

#folium.Marker(location=[center_lat_sf, center_lon_sf], icon=center_icon).add_to(map_sf)
#map_sf


map_sf_mean = folium.Map(location=[center_lat_sf, center_lon_sf], zoom_start=14)


coordinates = list(zip(df_sf_places['latitudes'], df_sf_places['longitudes']))
HeatMap(coordinates).add_to(map_sf_mean)

map_sf_mean.save("map_sf_mean.html")
map_sf_mean

In [497]:
folium.Marker(location=[center_lat_sf, center_lon_sf], icon=center_icon).add_to(map_sf_mean)
map_sf_mean

In [498]:
center_lat_sf, center_lon_sf


(37.7750333, -122.4217446)

In [47]:
bgl_map = folium.Map(location=[df_bgl_ll['latitude'][0], df_bgl_ll['longitude'][0]], zoom_start=14)

for i, row in df_bgl_ll.iterrows():
    folium.Marker(location=[row['latitude'], row['longitude']], popup=row['name']).add_to(bgl_map)

#bgl_map

In [64]:
stk_map = folium.Map(location=[df_stk_ll['latitude'][0], df_stk_ll['longitude'][0]], zoom_start=14)

for i, row in df_stk_ll.iterrows():
    folium.Marker(location=[row['latitude'], row['longitude']], popup=row['name']).add_to(stk_map)

#stk_map

In [226]:
#San Francisco:

    #Starbucks:
    
url = "https://api.foursquare.com/v3/places/search?ll=37.77%2C-122.42&radius=11000&chains=ab4c54c0-d68a-012e-5619-003048cad9da"
header = {"Authorization": token , "accept": "application/json" }

response = requests.get(url, headers=header)
lista_star_sf=response.json()
#lista_star_sf

In [277]:
latitudes = []
longitudes = []
names = []

for i in lista_star_sf['results']:
    latitude = i['geocodes']['main']['latitude']
    latitudes.append(latitude)
    longitude = i['geocodes']['main']['longitude']
    longitudes.append(longitude)
    name = i['chains'][0]['name'] + ', ' + i['location']['address']
    names.append(name)
    
#print(latitudes, longitudes, names)
df_star_sf = pd.DataFrame ({'names': names, 'latitudes': latitudes, 'longitudes': longitudes})
df_star_sf

,names,latitudes,longitudes
0,"Starbucks, 150 Van Ness Ave",37.777222,-122.419350
1,"Starbucks, 1100 Van Ness Ave",37.785916,-122.421011
2,"Starbucks, 2300 16th St",37.766862,-122.409564
3,"Starbucks, 1 Letterman Dr",37.799284,-122.449472
4,"Starbucks, 1700 Owens St",37.767121,-122.393738
5,"Starbucks, 49 Jefferson St",37.808488,-122.412881
6,"Starbucks, 299 Fremont St",37.788268,-122.393621
7,"Starbucks, 4094 18th St",37.761007,-122.434723
8,"Starbucks, 865 Market St",37.784117,-122.407356
9,"Starbucks, 3801 3rd St",37.741145,-122.387696


In [228]:
#Bangalore

    #Starbucks

url = url = "https://api.foursquare.com/v3/places/search?query=Starbucks&ll=12.96%2C77.58&radius=19000&limit=50"
headers = {"accept": "application/json", "Authorization": token}

response = requests.get(url, headers=headers)
lista_star_bgl = response.json()
lista_star_bgl['results'][4]['location']['formatted_address']

'Bangalore 560051, Karnātaka'

In [229]:
latitudes = []
longitudes = []
names = []

for i in lista_star_bgl['results']:
    latitude = i['geocodes']['main']['latitude']
    latitudes.append(latitude)
    longitude = i['geocodes']['main']['longitude']
    longitudes.append(longitude)
    name = i['name']+ ' ' + i['location']['formatted_address']
    names.append(name)
    
print(latitudes, longitudes, names)
df_star_bgl = pd.DataFrame ({'names': names, 'latitudes': latitudes, 'longitudes': longitudes})
df_star_bgl

[12.971404, 12.980862, 12.985943, 12.972406, 12.991671, 12.99165, 12.924554, 12.97225, 12.982138, 12.935201, 12.905996, 12.96996, 13.011109, 12.965622, 12.932558, 12.907647, 12.969994, 12.974645, 12.978288, 13.027641, 12.912103, 13.020864, 12.994205, 12.982376, 13.062913, 12.92141, 13.009025, 12.928374, 12.995844, 12.974602, 12.934738, 13.083196, 12.991692, 12.992449, 12.956944, 12.937444, 13.007699, 12.920194, 12.906018, 13.010978, 13.011414, 12.965622, 12.98545, 12.98719, 12.974541, 12.991404, 12.969659, 12.924445, 12.90604, 13.094566] [77.598237, 77.563662, 77.588422, 77.607285, 77.571129, 77.570896, 77.583892, 77.618544, 77.610405, 77.613483, 77.593685, 77.610179, 77.55521, 77.6418, 77.631816, 77.601363, 77.639902, 77.607332, 77.645026, 77.576251, 77.650807, 77.64296, 77.660513, 77.611209, 77.587117, 77.681297, 77.580299, 77.691852, 77.696502, 77.607208, 77.61148, 77.643432, 77.571191, 77.728716, 77.731079, 77.688775, 77.564718, 77.685123, 77.593747, 77.555183, 77.555215, 77.6418, 

,names,latitudes,longitudes
0,"Starbucks 38/1 Vittal Mallya Road, Bangalore 5...",12.971404,77.598237
1,"Starbucks Ground Floor, Lulu Global Malls (Bin...",12.980862,77.563662
2,"Starbucks Prestige Trade Tower, Bangalore 5600...",12.985943,77.588422
3,"Starbucks Brigade Road, Bangalore 972406, Karn...",12.972406,77.607285
4,"Starbucks Bangalore 560051, Karnātaka",12.991671,77.571129
5,"Starbucks Mantri Square, 103788, Karnataka",12.991650,77.570896
6,"Starbucks Bangalore 560011, Karnātaka",12.924554,77.583892
7,"Starbucks Kids Kemp Ramanashree Arcade, Ground...",12.972250,77.618544
8,"Starbucks Commercial Street, Bangalore 560001,...",12.982138,77.610405
9,"Starbucks Real Square Building, Ground Floor, ...",12.935201,77.613483
